# Initial setup

This analysis using ema is based on the work of Enayat A. Moallemi created on 22 May 2018 at the Fraunhofer ISI, Karlsruhe. In that instance, the analysis was done for the MATISSE model.

In [1]:
"""
Based on Enayat A. Moallemi version from May 2018

edited by Angela M. Rojas A. <angelara@student.unimelb.edu.au>

Created on February 2020

"""

import sys
import os

sys.path.append(r'C:\Users\angel\Documents\GitHub\gr4sp\experiments\EMAworkbench')

sys.path.append(r'C:\Users\angel\Documents\GitHub\gr4sp\experiments')

#from ema_workbench.analysis.plotting_util import BOXPLOT, KDE, VIOLIN
from ema_workbench.analysis.plotting import envelopes
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import matplotlib.cm as cm
import seaborn as sns
from ema_workbench.analysis import prim
#import pareto
import os
import glob
import numpy as np
import pandas as pd
import os
import glob
import datetime as DT




C:\Users\angel\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__



## Load the results

In [2]:
from ema_workbench import load_results

#results = load_results(r'C:\Users\angel\Documents\GitHub\gr4sp\experiments\data\gr4sp_2020-Feb-20-years.tar.gz')
#results = load_results(r'C:\Users\angel\Documents\GitHub\gr4sp\experiments\data\gr4sp_2020-Feb-07_EET_allV3.tar.gz')
#results = load_results(r'C:\Users\angel\Documents\GitHub\gr4sp\experiments\data\gr4sp_2020-Feb-25-LHS.tar.gz')
#results = load_results(r'C:/Users/angel/Documents/GitHub/gr4sp/experiments/simulationData/gr4sp_EET2020-Oct-05.tar.gz')
results = load_results(r'C:/Users/angel/Documents/GitHub/gr4sp/experiments/simulationData/gr4sp_SOBOL-HypoPast-2021-Jan-20.tar.gz')



In [3]:
experiments, outcomes = results
experiments.describe()

,consumption,domesticConsumptionPercentage,generationRolloutPeriod,importPriceFactor,includePublicallyAnnouncedGen,learningCurve,nameplateCapacityChangeBattery,nameplateCapacityChangeBrownCoal,nameplateCapacityChangeOcgt,nameplateCapacityChangeSolar,...,priceChangePercentageBattery,priceChangePercentageBrownCoal,priceChangePercentageOcgt,priceChangePercentageWater,priceChangePercentageWind,semiScheduleGenSpotMarket,semiScheduleMinCapMarketGen,solarUptake,technologicalImprovement,wholesaleTariffContribution
count,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,...,105000.000000,105000.000000,105000.000000,105000.000000,105000.000000,105000.0000,105000.000000,105000.000000,105000.000000,105000.000000
mean,2.000952,34.977381,5.495714,0.005952,0.500000,7.498095,0.003095,-0.013810,-0.022857,0.030238,...,-0.026429,0.005714,-0.019048,-0.018810,0.028810,9.0000,150.633333,2.000714,7.500000,27.493333
std,1.415062,8.947333,2.872209,29.151635,0.500002,4.609794,29.153775,29.154413,29.154383,29.152714,...,29.152857,29.153967,29.154059,29.155541,29.155884,0.8165,86.602028,1.414978,4.609794,10.387013
min,0.000000,20.000000,1.000000,-50.000000,0.000000,0.000000,-50.000000,-50.000000,-50.000000,-50.000000,...,-50.000000,-50.000000,-50.000000,-50.000000,-50.000000,8.0000,1.000000,0.000000,0.000000,10.000000
25%,1.000000,27.000000,3.000000,-25.000000,0.000000,3.750000,-25.000000,-25.000000,-25.000000,-25.000000,...,-25.000000,-25.000000,-25.000000,-25.000000,-25.000000,8.0000,75.750000,1.000000,3.750000,18.750000
50%,2.000000,35.000000,5.500000,0.000000,0.500000,7.500000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,9.0000,150.500000,2.000000,7.500000,27.500000
75%,3.000000,43.000000,8.000000,25.000000,1.000000,11.250000,25.000000,25.000000,25.000000,25.000000,...,25.000000,25.000000,25.000000,25.000000,25.000000,10.0000,225.250000,3.000000,11.250000,36.250000
max,4.000000,50.000000,10.000000,50.000000,1.000000,15.000000,50.000000,50.000000,50.000000,50.000000,...,50.000000,50.000000,50.000000,50.000000,50.000000,10.0000,300.000000,4.000000,15.000000,45.000000


Code to remove a Nan experiment if existent

Run this code to find the experiment that needs to be removed:

```python
	outcome_df = dict()
	for k in outcomes.keys():
	     outcome_df[k] = pd.DataFrame.from_dict(outcomes[k])
	    
	
	for k in outcomes.keys():
	    print(outcome_df[k][outcome_df[k].isna().any(axis=1)])

```

and then use the ``id`` of the experiment and remove it with the code below

In [4]:
outcomes_wo_nan = {}
for k in outcomes.keys():
    l1 = outcomes[k][0:41357]
    l2 = outcomes[k][41357+1:41379]
    l3 = outcomes[k][41379+1:]
    outcomes_wo_nan[k] = np.concatenate((l1,l2,l3))
outcomes = outcomes_wo_nan

## Load Energy Vulnerability

In [ ]:
# Energy vulnerability results
ev_results_df = pd.read_excel(r'C:/Users/angel/Documents/GitHub/gr4sp/experiments/notebookGr4sp/outputs/data/EV_all.xlsx', index_col=[0])

In [ ]:
# Duplicate results from 2040 into the years of 2040-2050

ev_results_df.insert(0, 1998, ev_results_df[2000])
ev_results_df.insert(0, 1999, ev_results_df[2000])
for t in range(2041,2051):
     ev_results_df[t] = ev_results_df[2040]
        
# Add it to outcomes data structure
outcomes['Energy Vulnerability Percentage'] = ev_results_df.values

In [ ]:
# Energy vulnerability results
ev_results_df_bau = pd.read_excel(r'C:/Users/angel/Documents/GitHub/gr4sp/experiments/notebookGr4sp/outputs/data/EV_oneScenario.xlsx', index_col=[0,1,2])

# Aggregate desagregated groups into a total parcentage
ev_bau = ev_results_df_bau.mean()

ev_bau_98_99 = pd.DataFrame(data=[ev_bau[2000],ev_bau[2000]],index=[1998,1999])
ev_bau_full = pd.concat([ev_bau_98_99, ev_bau], axis=0)

# Duplicate results from 2040 into the years of 2040-2050
for t in range(2041,2051):
     ev_bau_full.loc[t] = ev_bau_full.loc[2040]
        
outcomes['Energy Vulnerability Percentage'] = np.append(outcomes['Energy Vulnerability Percentage'],
                                                 [ev_bau_full.T.values[0]], axis=0)

## Load BAU

In [ ]:
bau_scenario_id = max(experiments['scenario']) + 1

new_row = pd.DataFrame({'annualCpi': 0.0233, 'annualInflation': 0.033, 'nameplateCapacityChangeBrownCoal': 0,
       'nameplateCapacityChangeWater': 0,'nameplateCapacityChangeOcgt': 0, 'consumption': 0, 'energyEfficiency': 0,
       'generationRolloutPeriod':1, 'generatorRetirement':0,
       'includePublicallyAnnouncedGen':0, 'learningCurve':0.05,
       'nonScheduleGenSpotMarket': 10, 'nonScheduleMinCapMarketGen':0.1,
       'priceChangePercentageBrownCoal':0, 'priceChangePercentageOcgt':0,
       'priceChangePercentageWater':0, 'priceChangePercentageWind':0, 'rooftopPV':7,
       'domesticConsumptionPercentage':0,
       'scheduleMinCapMarketGen':30, 'semiScheduleGenSpotMarket':8,
       'semiScheduleMinCapMarketGen':30, 'solarUptake':0,
       'technologicalImprovement':0.01, 'wholesaleTariffContribution':28.37, 'scenario': bau_scenario_id,
       'policy': "None", 'model': 'Gr4sp'}, index =[0]) 
# simply concatenate both dataframes 
experiments = pd.concat([experiments,new_row],sort=True).reset_index(drop = True) 

# rename none in policy column into constant value 0. PRIM fails otherwise.
if "None" in experiments["policy"].unique():
    experiments["policy"] = experiments["policy"].replace("None",int(0))

In [ ]:
#Results for BAU were generated with the 'light' setting in the model.
bau_results_df = pd.read_csv(r'C:/Users/angel/Documents/GitHub/gr4sp/experiments/simulationData/VICSimDataYearSummary_160221-BAU-1998-2051.csv')
bau_results_df.head()

In [ ]:
# #This cell adds BAU trajectory from 1998-2019. To add 1998 to 2050, comment this cell and uncomment the one below

# seed_bau = bau_scenario_id + 1
# outcomes['TIMEYear'] = np.append(outcomes['TIMEYear'],
#                                           [bau_results_df['Time (Year)'].values[:22]], axis=0)
# outcomes['consumptionYear'] = np.append(outcomes['consumptionYear'],
#                                                  [bau_results_df['Consumption (KWh) per household'].values[:22]], axis=0)
# outcomes['tariffsYear'] = np.append(outcomes['tariffsYear'],
#                                                  [bau_results_df['Avg Tariff (c/KWh) per household'].values[:22]], axis=0)
# outcomes['wholesalePriceYear'] = np.append(outcomes['wholesalePriceYear'],
#                                                  [bau_results_df['Primary Wholesale ($/MWh)'].values[:22]], axis=0)
# outcomes['GHGYear'] = np.append(outcomes['GHGYear'],
#                                                  [bau_results_df['GHG Emissions (tCO2-e) per household'].values[:22]], axis=0)
# outcomes['numConsumersYear'] = np.append(outcomes['numConsumersYear'],
#                                                  [bau_results_df['Number of Domestic Consumers (households)'].values[:22]], axis=0)
# outcomes['primarySpotProductionYear'] = np.append(outcomes['primarySpotProductionYear'],
#                                                  [bau_results_df['System Production Primary Spot'].values[:22]], axis=0)
# outcomes['secondarySpotProductionYear'] = np.append(outcomes['secondarySpotProductionYear'],
#                                                  [bau_results_df['System Production Secondary Spot'].values[:22]], axis=0)
# outcomes['offSpotProductionYear'] = np.append(outcomes['offSpotProductionYear'],
#                                                  [bau_results_df['System Production Off Spot'].values[:22]], axis=0)
# outcomes['renewableContributionYear'] = np.append(outcomes['renewableContributionYear'],
#                                                  [bau_results_df['Percentage Renewable Production'].values[:22]], axis=0)
# outcomes['rooftopPVProductionYear'] = np.append(outcomes['rooftopPVProductionYear'],
#                                                  [bau_results_df['System Production Rooftop PV'].values[:22]], axis=0)
# outcomes['coalProductionYear'] = np.append(outcomes['coalProductionYear'],
#                                                  [bau_results_df['System Production Coal'].values[:22]], axis=0)
# outcomes['windProductionYear'] = np.append(outcomes['windProductionYear'],
#                                                  [bau_results_df['System Production Wind'].values[:22]], axis=0)
# outcomes['gasProductionYear'] = np.append(outcomes['gasProductionYear'],
#                                                  [bau_results_df['System Production Gas'].values[:22]], axis=0)
# outcomes['solarProductionYear'] = np.append(outcomes['solarProductionYear'],
#                                                  [bau_results_df['System Production Solar'].values[:22]], axis=0)   
# outcomes['waterProductionYear'] = np.append(outcomes['waterProductionYear'],
#                                                  [bau_results_df['System Production Water'].values[:22]], axis=0)
# outcomes['BatteryProductionYear'] = np.append(outcomes['BatteryProductionYear'],
#                                                  [bau_results_df['System Production Battery'].values[:22]], axis=0)
# outcomes['numActorsYear'] = np.append(outcomes['numActorsYear'],
#                                                  [bau_results_df['Number of Active Actors'].values[:22]], axis=0)
# outcomes['primaryUnmetDemandMwh'] = np.append(outcomes['primaryUnmetDemandMwh'],
#                                                  [bau_results_df['Primary Total Unmet Demand (MWh)'].values[:22]], axis=0)
# outcomes['primaryUnmetDemandHours'] = np.append(outcomes['primaryUnmetDemandHours'],
#                                                  [bau_results_df['Primary Total Unmet Demand (Hours)'].values[:22]], axis=0)
# outcomes['primaryUnmetDemandDays'] = np.append(outcomes['primaryUnmetDemandDays'],
#                                                  [bau_results_df['Primary Total Unmet Demand (Days)'].values[:22]], axis=0)
# outcomes['primaryMaxUnmetDemandMwhPerHour'] = np.append(outcomes['primaryMaxUnmetDemandMwhPerHour'],
#                                                  [bau_results_df['Primary Max Unmet Demand Per Hour (MWh)'].values[:22]], axis=0)
# outcomes['secondaryUnmetDemandMwh'] = np.append(outcomes['secondaryUnmetDemandMwh'],
#                                                  [bau_results_df['Secondary Total Unmet Demand (MWh)'].values[:22]], axis=0)
# outcomes['secondaryUnmetDemandHours'] = np.append(outcomes['secondaryUnmetDemandHours'],
#                                                  [bau_results_df['Secondary Total Unmet Demand (Hours)'].values[:22]], axis=0)
# outcomes['secondaryUnmetDemandDays'] = np.append(outcomes['secondaryUnmetDemandDays'],
#                                                  [bau_results_df['Secondary Total Unmet Demand (Days)'].values[:22]], axis=0)
# outcomes['secondaryMaxUnmetDemandMwhPerHour'] = np.append(outcomes['secondaryMaxUnmetDemandMwhPerHour'],
#                                                  [bau_results_df['Secondary Max Unmet Demand Per Hour (MWh)'].values[:22]], axis=0)
                                          
# outcomes['seedExperimentCsv'] = np.append(outcomes['seedExperimentCsv'], [seed_bau], axis=0)


In [ ]:
seed_bau = bau_scenario_id + 1
outcomes['TIMEYear'] = np.append(outcomes['TIMEYear'],
                                          [bau_results_df['Time (Year)'].values], axis=0)
outcomes['consumptionYear'] = np.append(outcomes['consumptionYear'],
                                                 [bau_results_df['Consumption (KWh) per household'].values], axis=0)
outcomes['tariffsYear'] = np.append(outcomes['tariffsYear'],
                                                 [bau_results_df['Avg Tariff (c/KWh) per household'].values], axis=0)
outcomes['wholesalePriceYear'] = np.append(outcomes['wholesalePriceYear'],
                                                 [bau_results_df['Primary Wholesale ($/MWh)'].values], axis=0)
outcomes['GHGYear'] = np.append(outcomes['GHGYear'],
                                                 [bau_results_df['GHG Emissions (tCO2-e) per household'].values], axis=0)
outcomes['numConsumersYear'] = np.append(outcomes['numConsumersYear'],
                                                 [bau_results_df['Number of Domestic Consumers (households)'].values], axis=0)
outcomes['primarySpotProductionYear'] = np.append(outcomes['primarySpotProductionYear'],
                                                 [bau_results_df['System Production Primary Spot'].values], axis=0)
outcomes['secondarySpotProductionYear'] = np.append(outcomes['secondarySpotProductionYear'],
                                                 [bau_results_df['System Production Secondary Spot'].values], axis=0)
outcomes['offSpotProductionYear'] = np.append(outcomes['offSpotProductionYear'],
                                                 [bau_results_df['System Production Off Spot'].values], axis=0)
outcomes['renewableContributionYear'] = np.append(outcomes['renewableContributionYear'],
                                                 [bau_results_df['Percentage Renewable Production'].values], axis=0)
outcomes['rooftopPVProductionYear'] = np.append(outcomes['rooftopPVProductionYear'],
                                                 [bau_results_df['System Production Rooftop PV'].values], axis=0)
outcomes['coalProductionYear'] = np.append(outcomes['coalProductionYear'],
                                                 [bau_results_df['System Production Coal'].values], axis=0)
outcomes['windProductionYear'] = np.append(outcomes['windProductionYear'],
                                                 [bau_results_df['System Production Wind'].values], axis=0)
outcomes['gasProductionYear'] = np.append(outcomes['gasProductionYear'],
                                                 [bau_results_df['System Production Gas'].values], axis=0)
outcomes['solarProductionYear'] = np.append(outcomes['solarProductionYear'],
                                                 [bau_results_df['System Production Solar'].values], axis=0)   
outcomes['waterProductionYear'] = np.append(outcomes['waterProductionYear'],
                                                 [bau_results_df['System Production Water'].values], axis=0)
outcomes['BatteryProductionYear'] = np.append(outcomes['BatteryProductionYear'],
                                                 [bau_results_df['System Production Battery'].values], axis=0)
outcomes['numActorsYear'] = np.append(outcomes['numActorsYear'],
                                                 [bau_results_df['Number of Active Actors'].values], axis=0)
outcomes['primaryUnmetDemandMwh'] = np.append(outcomes['primaryUnmetDemandMwh'],
                                                 [bau_results_df['Primary Total Unmet Demand (MWh)'].values], axis=0)
outcomes['primaryUnmetDemandHours'] = np.append(outcomes['primaryUnmetDemandHours'],
                                                 [bau_results_df['Primary Total Unmet Demand (Hours)'].values], axis=0)
outcomes['primaryUnmetDemandDays'] = np.append(outcomes['primaryUnmetDemandDays'],
                                                 [bau_results_df['Primary Total Unmet Demand (Days)'].values], axis=0)
outcomes['primaryMaxUnmetDemandMwhPerHour'] = np.append(outcomes['primaryMaxUnmetDemandMwhPerHour'],
                                                 [bau_results_df['Primary Max Unmet Demand Per Hour (MWh)'].values], axis=0)
outcomes['secondaryUnmetDemandMwh'] = np.append(outcomes['secondaryUnmetDemandMwh'],
                                                 [bau_results_df['Secondary Total Unmet Demand (MWh)'].values], axis=0)
outcomes['secondaryUnmetDemandHours'] = np.append(outcomes['secondaryUnmetDemandHours'],
                                                 [bau_results_df['Secondary Total Unmet Demand (Hours)'].values], axis=0)
outcomes['secondaryUnmetDemandDays'] = np.append(outcomes['secondaryUnmetDemandDays'],
                                                 [bau_results_df['Secondary Total Unmet Demand (Days)'].values], axis=0)
outcomes['secondaryMaxUnmetDemandMwhPerHour'] = np.append(outcomes['secondaryMaxUnmetDemandMwhPerHour'],
                                                 [bau_results_df['Secondary Max Unmet Demand Per Hour (MWh)'].values], axis=0)

outcomes['seedExperimentCsv'] = np.append(outcomes['seedExperimentCsv'], [seed_bau], axis=0)


## General parameters

In [5]:
outcomes['PrimarySpot-WholesalePriceYear'] = outcomes['wholesalePriceYear']

outcomes_to_show = ['consumptionYear', 'tariffsYear', 'PrimarySpot-WholesalePriceYear', 'GHGYear', 
                    'numConsumersYear', 'primarySpotProductionYear', 'secondarySpotProductionYear', 
                    'offSpotProductionYear', 'renewableContributionYear', 'rooftopPVProductionYear', 
                    'coalProductionYear', 'waterProductionYear', 'windProductionYear', 'gasProductionYear', 
                    'solarProductionYear', 'BatteryProductionYear', 'numActorsYear', 'primaryUnmetDemandMwh', 
                    'primaryUnmetDemandHours', 'primaryUnmetDemandDays', 'primaryMaxUnmetDemandMwhPerHour', 
                    'secondaryUnmetDemandMwh', 'secondaryUnmetDemandHours', 'secondaryUnmetDemandDays', 
                    'secondaryMaxUnmetDemandMwhPerHour']

# 23 levers/uncertainties selected after EET (last test July 2020)
# 25 uncertainties selected aftet EET including change on domestic demand and unmet demand output (August 2020)
uncertainties = experiments.columns[:-3]

#from startYear
startYear = 2000
startYearShift = (startYear - 1998)

time = outcomes['TIMEYear'][0, startYearShift:]

points_of_interest = [*range(2020,2051,10)] 
#points_of_interest = [*range(2000,2019,5)] 

points_of_interest = [ x - startYear for x in points_of_interest]

# Filter outcomes taking into account the startYear, and removing Seed Experiment info. 
# Seed it is useful if I want to retrieve more info from the csv files, or redo the experiments with the
# simulation parameters
outcomes_filtered = {k:v[:, startYearShift:] for k,v in outcomes.items() if k != 'seedExperimentCsv'}


TypeError: list indices must be integers or slices, not tuple

## Aggregate Household data

GHG emissions and Consumption per household type converted to total sector


In [ ]:
ooi = 'GHGYear'
data_GHG_household = outcomes_filtered[ooi]

In [ ]:
""" Converts household data in tCO2e into total residential data in MtCO2e"""
data_GHG_residential = (data_GHG_household * outcomes_filtered['numConsumersYear']) / 1000000.0
data_GHG_residential_bau = (bau_results_df['GHG Emissions (tCO2-e) per household'] * bau_results_df['Number of Domestic Consumers (households)']) / 1000000.0

""" Applies scaling factor used in simulation, so that data represents total emissions, not only residential (30%)"""
data_GHG_total = data_GHG_residential / 0.3
data_GHG_total_bau = data_GHG_residential_bau / 0.3


"""Overwrite GHG with the data of interest"""
outcomes_filtered[ooi] = data_GHG_total
bau_results_df[ooi] = data_GHG_total_bau

In [ ]:
ooi = 'consumptionYear'
data_consumption_household = outcomes_filtered[ooi]

In [ ]:
""" Converts household data in KWh into total residential data in MWh"""
data_consumption_residential = (data_consumption_household * outcomes_filtered['numConsumersYear']) / 1000.0

""" Applies scaling factor used in simulation, so that data represents total consumption, not only residential"""
data_consumption_total = data_consumption_residential / 0.3

"""Overwrite Consumption with the data of interest"""
outcomes_filtered[ooi] = data_consumption_total

# Target functions

In [ ]:
def target_always_after(outcomes,ooi,targetYear,targetVal,above):
    data = outcomes[ooi]
    num_exp = len(data)
    achieved = np.zeros(num_exp, dtype=bool)

    for i in range(num_exp):
        achieved[i] = False
        exp_run = data[i]
        for j in range( len(exp_run) ):
            if (startYear + j) >= targetYear:
                if above is True:
                    achieved[i] = exp_run[j] >= targetVal
                else:
                    achieved[i] = exp_run[j] < targetVal
                    
                #If the target is not achieved after the TargetYear, then mark experiment as False
                if achieved[i] == False:
                    break
    return achieved

def target_at_year(outcomes,ooi,targetYear,targetVal,above):
    data = outcomes[ooi]
    num_exp = len(data)
    achieved = np.zeros(num_exp, dtype=bool)

    for i in range(num_exp):
        achieved[i] = False
        exp_run = data[i]
        
        idx = targetYear - startYear
        if above is True:
            achieved[i] = exp_run[idx] >= targetVal
        else:
            achieved[i] = exp_run[idx] < targetVal                        
                
    return achieved

# Scenario discovery

## Open Exploration: Visual Analysis


Truncate experiments and outcomes for a selected policy and a selected outcome over mean in time series

### GHG emissions

In [ ]:
#Subset of levers/uncertainties
#selected_levers = list(['rooftopPV', 'annualCpi','capacityFactorChangeBrownCoal','capacityFactorChangeWater','semiScheduleMinCapMarketGen'])    
selected_uncertainties = uncertainties

ooi = 'GHGYear'

data = outcomes_filtered[ooi]


num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

The following section uses PRIM, feature scoring and dimensional stacking to find the cases that represent futures in three main groups: Low Carbon, Just Transition and Sustainable Transition.

It uses targets (e.g. cases for which emissions are the lowest at year x) to filter out the cases and find out the corresponding input variables and ranges for the cases to happen.

The first case is on GHG emissions. To find out the targets for exploration, first let's analyse the envelope diagram (see below) showing how the transition may unfold:

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density 

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest, density = Density.KDE, grouping_specifiers=None, 
                                 titles={ooi}, ylabels={ooi: "MtCO2e"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])

#Add BAU in the plot with a gray wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

## Show labels
# labels = ax_env['F']['main plot'].get_xticklabels()
# for l in labels:
#     print (l.get_text)

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')
    

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density 

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest, density = Density.HIST, grouping_specifiers=None,
                                 titles={ooi}, ylabels={ooi: "MtCO2e"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])

#Add BAU in the plot with a gray red line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-')

## Show labels
# labels = ax_env['F']['main plot'].get_xticklabels()
# for l in labels:
#     print (l.get_text)

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)
#plt.legend()

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')


### PrimarySpot Wholesale Price

In [ ]:
# Subset Sobol levers
#selected_levers = list(['rooftopPV','annualCpi', 'generationRolloutPeriod', 'generatorRetirement','capacityFactorChangeBrownCoal','semiScheduleMinCapMarketGen'])    

select_uncertainties = uncertainties
ooi = 'PrimarySpot-WholesalePriceYear'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, select_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density 

# fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
#                                  points_in_time=points_of_interest,  density = Density.KDE,
#                                  grouping_specifiers=None , titles={ooi}, ylabels={ooi: "$/MWh"}, legend=True,
#                                  fill=False, plot_type=PlotType.ENV_LIN)

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "$/MWh"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)
#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

# fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
#                                  points_in_time=points_of_interest,  density = Density.HIST,
#                                  grouping_specifiers=None , titles={ooi}, ylabels={ooi: "$/MWh"}, legend=True,
#                                  fill=True, plot_type=PlotType.ENVELOPE)

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.HIST,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "$/MWh"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')



### Tariffs

In [ ]:
# Subset Sool levers/uncertainties
selected_uncertainties = uncertainties

#selected_levers = levers
ooi = 'tariffsYear'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

: 20-25% by 2025 on 2017. tariffs in 2017: ~21 c/KWh

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "c/KWh"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )


fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.HIST,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "c/KWh"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )


fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')


### Renewable Energies in the System

In [ ]:
# Subset Sool levers/uncertainties

selected_uncertainties = uncertainties

ooi = 'renewableContributionYear'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "% Renewable"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.HIST,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "% Renewable"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Primary Unmet demand in MWh 

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'primaryUnmetDemandMwh'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "MWh"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "MWh"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Primary unmet demand (hours)

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'primaryUnmetDemandHours'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "hours"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "hours"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Primary unmet demand (days)

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'primaryUnmetDemandDays'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "days"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "days"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Primary max unmet demand per hour

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'primaryMaxUnmetDemandMwhPerHour'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "max MWh in an hour"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "max MWh in an hour"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )


fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Secondary market unmet demand MWh

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'secondaryUnmetDemandMwh'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "MWh"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "MWh"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_hypopast_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

### Unmet Demand - Secondary market unmet demand (hours)

### Unmet Demand - Secondary unmet demand (days)

### Unmet Demand - Secondary market max unmet demand per hour

: 20-25% by 2025 on 2017. tariffs in 2017: ~21 c/KWh

As the envelope shows, there are two main groups at the first years of analysis where GHG emissions either increase or decrease. However, as time passes, GHG emissions can vary widely. 
Taking the first year of analysis, year 2020, we can investigate the cases where GHG emissions per year per household fall below 30.89 MtonsCO2-e. This is 20% lower than the emissions in BAU for 2005, which were 38.62 MtCO2-e

## Directed Search: Prim - Low Carbon Scenario
* Low carbon emissions is the objective. Therefore, low emissions scenarios, with a high generation autonomy (in primary and secondary markets) and high percentage of renewable energy in the system are going to be explored. 

This section has two parts: the hypothetical past and the future pathways. 
1. Hypothetical past: Fins out what Victoria could have done to get into the lower cluster of pathways with low carbon emissions.

2. Victoria's Climate Change Act 2017: “These targets build on the progress that Victoria is already making; emissions are projected to fall to 18% below 2005 levels in 2020.”
The Panel recommends interim targets of:
•	32-39% below 2005 levels in 2025
•	45-60% below 2005 levels in 2030

In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);

x = exp
"""
Paris trajectory to 1.5 C
Filter by GHG (below 39% value of 2005 by 2025)
"""    
ooi = 'GHGYear'

reference = bau_results_df[bau_results_df['Time (Year)'] == 2005]['GHGYear'].values

reduction = 0.39
ylc_e1 = target_at_year( outcomes_filtered, ooi, 2025, reference - (reference * reduction), False) #False is below target

reduction = 0.60
ylc_e2 = target_at_year( outcomes_filtered, ooi, 2030, reference - (reference * reduction), False) #False is below target

reduction = 0.7
ylc_e3 = target_at_year( outcomes_filtered, ooi, 2050, reference - (reference * reduction), False) #False is below target

ylc_e = ylc_e1 &  ylc_e2 #& ylc_e3 

"""
Filter by renewableContributionYear (above 50%)
"""

ooi = 'renewableContributionYear'    

#y = out_avg_ooi > 20

ylc_re1 = target_at_year( outcomes_filtered, ooi, 2030, 0.40, True) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target
ylc_re2 = target_at_year( outcomes_filtered, ooi, 2050, 0.50, True) #False is below target


ylc_re= ylc_re1 & ylc_re2

""""
Filter by GA of Primary Wholesale Market (below 70% value of 2019, which is the year with Max unmet demand per hour)
"""

ooi = 'primaryUnmetDemandMwh'    

#y = out_avg_ooi > 20

ylc_gap1 = target_at_year( outcomes_filtered, ooi, 2040, 41346.343, False) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target


# """
# Combine Filters
# """
y = ylc_e & ylc_re & ylc_gap1

"""
Run PRIM
"""
prim_alg = prim.Prim(x, y, threshold=0.1, threshold_type=prim.ABOVE)
box1 = prim_alg.find_box()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
#%matplotlib inline
selected_box = 22

box_1 = box1.inspect(selected_box)
box_1 = box1.inspect(selected_box, style='graph')


"The quasi-p-value test, essentially a one sided binomial test, is an estimate of the likelihood that a given uncertainty is included in the definition of the subspace purely by chance." (Bryant and Lempert, 2010)
"
“qp-value” test thus answers the question: what is the probability that Tpoints drawn from the above binomial distribution would have H or more high valued points? When the ratio $H_{-j}$/$T_{-j}$ is close to H/T this number is high, the additional contribution of parameter $r_j$ is low, and thus possibly due to chance. The opposite is the case when H/T is much larger than  $H_{-j}$/$T_{-j}$." (Bryant and Lempert, 2010)

In [ ]:
box1.select(selected_box)
fig = box1.show_pairs_scatter(selected_box)

plt.show()


### BAU vs LCT

In [ ]:
lct_results_df = pd.read_csv(r'C:/Users/angel/Documents/GitHub/gr4sp/csv/LCTVIC/LCTVICSimDataYearSummary_seed_1613443578132.csv')

""" Converts household data in tCO2e into total residential data in MtCO2e"""
data_GHG_residential_lct = (lct_results_df['GHG Emissions (tCO2-e) per household'] * lct_results_df['Number of Domestic Consumers (households)']) / 1000000.0

""" Applies scaling factor used in simulation, so that data represents total emissions, not only residential (30%)"""
data_GHG_total_lct = data_GHG_residential_lct / 0.3

"""Overwrite GHG with the data of interest"""
lct_results_df['GHGYear'] = data_GHG_total_lct

In [ ]:
plt.figure(figsize=(18,10))
years = [DT.datetime.strptime(str(int(date)),'%Y') for date in bau_results_df['Time (Year)'].values]
years =  bau_results_df['Time (Year)'].values
plt.plot( years, bau_results_df['GHGYear'], markerfacecolor='lightblue', color='lightblue', linewidth=4, label='GHG Emissions (MtCO2-e) BAU')
plt.plot( years, lct_results_df['GHGYear'], markerfacecolor='darkgoldenrod',  color='burlywood', linewidth=4, label='GHG Emissions (MtCO2-e) LCT')

# Add dot and corresponding text
reference = bau_results_df[bau_results_df['Time (Year)'] == 2005]['GHGYear'].values

reduction = 0.39

plt.axhline(  (reference - (reference * reduction)) ,ls='--',color='red')
plt.text(years[0], reference - (reference * reduction)+2, f'2025 Target: {((reference - (reference * reduction))[0]):.2f}',color='red')
plt.plot(2025, (reference - (reference * reduction)), 'ro')


reduction = 0.60

plt.axhline(  (reference - (reference * reduction)) ,ls='--',color='green')
plt.text(years[0], reference - (reference * reduction)+2, f'2030 Target: {((reference - (reference * reduction))[0]):.2f}',color='green')
plt.plot(2030, (reference - (reference * reduction)), 'go')

reduction = 0.7
plt.axhline(  (reference - (reference * reduction)) ,ls='--',color='blue')
plt.text(years[0], reference - (reference * reduction)+2, f'2050 Target: {((reference - (reference * reduction))[0]):.2f}',color='blue')
plt.plot(2050, (reference - (reference * reduction)), 'bo')

#plt.grid( linestyle='-', linewidth=1)
plt.xlabel('Year', size=16)
plt.ylabel('GHGe', size=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
plt.legend(prop={'size':16})

In [ ]:
plt.figure(figsize=(18,10))
years = [DT.datetime.strptime(str(int(date)),'%Y') for date in bau_results_df['Time (Year)'].values]
years =  bau_results_df['Time (Year)'].values
plt.plot( years, bau_results_df['Percentage Renewable Production'], markerfacecolor='lightblue', color='lightblue', linewidth=4, label='Percentage Renewable Production BAU')
plt.plot( years, lct_results_df['Percentage Renewable Production'], markerfacecolor='darkgoldenrod',  color='burlywood', linewidth=4, label='Percentage Renewable Production LCT')

# Add dot and corresponding text

target = 0.40

plt.axhline(  target ,ls='--',color='green')
plt.text(years[0], target+0.01, f'2030 Target: {target:.2f}',color='green')
plt.plot(2030, target, 'go')

target = 0.5
plt.axhline(  target ,ls='--',color='blue')
plt.text(years[0], target+0.01, f'2050 Target: {target:.2f}',color='blue')
plt.plot(2050, target, 'bo')

#plt.grid( linestyle='-', linewidth=1)
plt.xlabel('Year', size=16)
plt.ylabel('renewableContributionYear', size=16)
plt.xticks(size = 14)
plt.yticks(size = 14)
plt.legend(prop={'size':16})

### Feature Scoring

In [ ]:
from EMAworkbench.ema_workbench.analysis import feature_scoring

fs, alg = feature_scoring.get_ex_feature_scores(x, y, mode=feature_scoring.RuleInductionType.CLASSIFICATION)
fs.sort_values(ascending=False, by=1)



In [ ]:
x = exp
y_all = {}

#plot using ALL simulations
#y_all['GHGYear'] = data

#plot using average of ALL simulations
#y_all['GHGYear'] = out_avg_ooi

#Plot using only target simulations
y_all['GHGYear'] = y

fig, ax = plt.subplots(figsize=(10,10))
fs = feature_scoring.get_feature_scores_all(x, y_all)
sns.heatmap(fs, cmap='viridis', annot=True, ax=ax)
plt.show()

### Dimensional Stacking


In [ ]:

from EMAworkbench.ema_workbench.analysis import dimensional_stacking

x = exp
dimensional_stacking.create_pivot_plot(x,y, 1, nbins=3)
plt.show()



## Direct search: PRIM - Just Transition
Includes trajectories with low tarifffs, EV and good autonomy of secondary markets

In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);

x = exp

"""
Filter by Energy Vulnerability (below 70% of 2017 value)
"""
ooi = 'Energy Vulnerability Percentage'    

#y = out_avg_ooi > 20

yjt_ev1= target_at_year( outcomes_filtered, ooi, 2030, 0.3, False) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target


""""
Filter by GA of Primary Wholesale Market (below 70% value of 2019, which is the year with Max unmet demand per hour)
"""

ooi = 'secondaryUnmetDemandMwh'    

#y = out_avg_ooi > 20

yjt_gas1 = target_at_year( outcomes_filtered, ooi, 2030, 4223.991784, False) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target


# """
# Combine Filters
# """
y = yjt_ev1 #& yjt_gas1

"""
Run PRIM
"""
prim_alg = prim.Prim(x, y, threshold=0.1, threshold_type=prim.ABOVE)
box1 = prim_alg.find_box()


In [ ]:
box1.inspect_tradeoff()

We can see that the targets are __not ambitious enough__, and is sufficient to __delay slightly the rollout of renewables__, and **avoid the earliest retirement of brown coal**

In [ ]:
%matplotlib inline
box_1 = box1.inspect(3)
box_1 = box1.inspect(3, style='graph')

### Feature Scoring

In [ ]:
from EMAworkbench.ema_workbench.analysis import feature_scoring

fs, alg = feature_scoring.get_ex_feature_scores(x, y, mode=feature_scoring.RuleInductionType.CLASSIFICATION)
fs.sort_values(ascending=False, by=1)


In [ ]:
x = exp
y_all = {}

#plot using ALL simulations
#y_all['GHGYear'] = data

#plot using average of ALL simulations
#y_all['GHGYear'] = out_avg_ooi

#Plot using only target simulations
y_all['WholesalePrice'] = y

fig, ax = plt.subplots(figsize=(5,10))
fs = feature_scoring.get_feature_scores_all(x, y_all)
sns.heatmap(fs, cmap='viridis', annot=True, ax=ax)
plt.show()

### Dimensional stacking

In [ ]:

from EMAworkbench.ema_workbench.analysis import dimensional_stacking

x = exp
dimensional_stacking.create_pivot_plot(x,y, 1, nbins=3)
plt.show()



### A more ambitious target
Wholesale prices: 50% by 2025 on 2017 levels
Spot price in 2017 ~103 $/MWh (OpenNem) in simulation BAU VIC 48.05. In 2025 should be below 24.02 (50%)

In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);
 
x = exp
#y = out_avg_ooi > 80

y = target_at_year( outcomes_filtered, ooi, 2040, 50, False) #False is below target

prim_alg = prim.Prim(x, y, threshold=.35, threshold_type=prim.ABOVE)
box1 = prim_alg.find_box()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
%matplotlib inline
box_1 = box1.inspect(3)
box_1 = box1.inspect(13, style='graph')


In [ ]:
box1.select(13)
fig = box1.show_pairs_scatter(13)

plt.show()

### Feature Scoring

In [ ]:
from EMAworkbench.ema_workbench.analysis import feature_scoring

fs, alg = feature_scoring.get_ex_feature_scores(x, y, mode=feature_scoring.RuleInductionType.CLASSIFICATION)
fs.sort_values(ascending=False, by=1)


In [ ]:
x = exp
y_all = {}

#plot using ALL simulations
#y_all['GHGYear'] = data

#plot using average of ALL simulations
#y_all['GHGYear'] = out_avg_ooi

#Plot using only target simulations
y_all['WholesalePrice'] = y

fig, ax = plt.subplots(figsize=(5,10))
fs = feature_scoring.get_feature_scores_all(x, y_all)
sns.heatmap(fs, cmap='viridis', annot=True, ax=ax)
plt.show()

### Dimensional stacking

In [ ]:

from EMAworkbench.ema_workbench.analysis import dimensional_stacking

x = exp
dimensional_stacking.create_pivot_plot(x,y, 1, nbins=3)
plt.show()



## Tariffs

#### Filter trajectories away from the BAU

In [ ]:
filter_below = target_at_year( outcomes_filtered, ooi, 2000, 30, False) #True means above target
#outcomes_filtered = {k:v[:, startYearShift:] for k,v in outcomes.items() if k != 'seedExperimentCsv'}

outcomes_bau_cluster = {k:v[filter_below, :] for k,v in outcomes_filtered.items()}

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density 

fig, ax_env = multiple_densities(experiments, outcomes_bau_cluster, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest, density = Density.KDE, grouping_specifiers=None, 
                                 titles={ooi}, ylabels={ooi: "c/KWh"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])

#Add BAU in the plot with a gray wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

## Show labels
# labels = ax_env['F']['main plot'].get_xticklabels()
# for l in labels:
#     print (l.get_text)

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', 'filtered_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

# fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
#                                  points_in_time=points_of_interest,  density = Density.HIST,
#                                  grouping_specifiers=None , titles={ooi}, ylabels={ooi: "$/MWh"}, legend=True,
#                                  fill=True, plot_type=PlotType.ENVELOPE)

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.HIST,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "c/KWh"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=3 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')



In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);

x = exp
"""
Filter by GHG
"""    
ooi = 'GHGYear'
#y = (out_avg_ooi < 4.)
#y = target_at_year( outcomes_filtered, ooi, 2030, 24, False) #False is below target
y1 = target_at_year( outcomes_filtered, ooi, 2030, 25, False) #False is below target

"""
Filter by Wholesale price
"""

ooi = 'wholesalePriceYear'    

#y = out_avg_ooi > 20

y2 = target_at_year( outcomes_filtered, ooi, 2030, 50, False) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target

"""
Filter by Tariff
"""

ooi = 'tariffsYear'    

#y = out_avg_ooi > 20

y3 = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target
#y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target

"""
Combine Filters
"""
y = y1 & y2 & y3

"""
Run PRIM
"""
prim_alg = prim.Prim(x, y, threshold=0.1, threshold_type=prim.ABOVE)
box1 = prim_alg.find_box()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
%matplotlib inline
box_1 = box1.inspect(32)
box_1 = box1.inspect(32, style='graph')

### Dimensional stacking

In [ ]:

from EMAworkbench.ema_workbench.analysis import dimensional_stacking

x = exp
dimensional_stacking.create_pivot_plot(x,y, 2, nbins=3)
plt.show()



In [ ]:
box1.select(12)
fig = box1.show_pairs_scatter(12)
plt.show()

In [ ]:
## Primary Spot Production

In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);
 
"""
Filter by GHG
"""    
#ooi = 'GHGYear'
#y = (out_avg_ooi < 4.)
#y = target_at_year( outcomes_filtered, ooi, 2030, 24, False) #False is below target
#y1 = target_at_year( outcomes_filtered, ooi, 2020, 38, False) #False is below target

"""
Filter by Tariff
"""

ooi = 'tariffsYear'    
x = exp
#y = out_avg_ooi > 20

#y2 = target_at_year( outcomes_filtered, ooi, 2030, 50, True) #False is below target
y = target_at_year( outcomes_filtered, ooi, 2030, 21, False) #False is below target

"""
Combine Filters
"""
#y = y1 & y2

"""
Run PRIM
"""
prim_alg = prim.Prim(x, y, threshold=0.5, threshold_type=prim.ABOVE)
box1 = prim_alg.find_box()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
%matplotlib inline
box_1 = box1.inspect(11)
box_1 = box1.inspect(11, style='graph')

### Dimensional stacking

In [ ]:

from EMAworkbench.ema_workbench.analysis import dimensional_stacking

x = exp
dimensional_stacking.create_pivot_plot(x,y, 1, nbins=3)
plt.show()



In [ ]:
box1.select(11)
fig = box1.show_pairs_scatter(11)
plt.show()

In [ ]:
# Subset Sool levers
selected_uncertainties = uncertainties

ooi = 'secondaryMaxUnmetDemandMwhPerHour'
data = outcomes_filtered[ooi]

num_exp = len(data)
out_avg_ooi = np.zeros(num_exp)

for i in range(num_exp):
    out_avg_ooi[i] = data[i].mean()

exp = pd.DataFrame(experiments)

exp = exp.loc[ :, selected_uncertainties] 

### Open Exploration: Visual Analysis

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "max MWh in an hour"}, legend=True,
                                 fill=False, plot_type=PlotType.ENV_LIN)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a black wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'k-',linewidth=5 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_envelope_line_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis.plotting import multiple_densities, PlotType
from EMAworkbench.ema_workbench.analysis.plotting_util import Density

fig, ax_env = multiple_densities(experiments, outcomes_filtered, outcomes_to_show = ooi,
                                 points_in_time=points_of_interest,  density = Density.KDE,
                                 grouping_specifiers=None , titles={ooi}, ylabels={ooi: "max MWh in an hour"}, legend=True,
                                 fill=True, plot_type=PlotType.ENVELOPE)

#Change x labels to years
ax_env[ooi]['main plot'].set_xticklabels(time[range(0,len(time),10)])
#Add BAU in the plot with a red wide line
ax_env[ooi]['main plot'].plot(outcomes_filtered[ooi][-1], 'r-',linewidth=2 )

fig[0].suptitle("{}".format(ooi), fontsize=15)
fig[0].set_size_inches(18.5, 10.5)

plt.savefig('{}/fig{}.png'.format(r'C:\\Users\\angel\\Documents\\GitHub\\gr4sp\\experiments\\notebookGr4sp\\outputs\\figs', '_envelope_density_%s'%(ooi)),
                dpi=300, bbox_inches='tight')

In [ ]:
from EMAworkbench.ema_workbench.analysis import prim
from EMAworkbench.ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO);
 
x = exp
#y = out_avg_ooi > 40000000
y = target_at_year( outcomes_filtered, ooi, 2035, 40000000, False) #False is below target

prim_alg = prim.Prim(x, y, threshold=0.3)
box1 = prim_alg.find_box()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
%matplotlib inline
box_1 = box1.inspect(2)
box_1 = box1.inspect(2, style='graph')

In [ ]:
box1.select(2)
fig = box1.show_pairs_scatter(2)
plt.show()